In [1]:
import pandas as pd
from siuba import *

In [2]:
scolist = pd.read_excel('gs://calitp-analytics-data/data-analyses/ntd/RTPAs Contact Information FY 2021-22.xlsx')

In [3]:
from siuba.experimental.pivot import pivot_wider, pivot_longer
from siuba import Fx

In [4]:
fns = (scolist 
        >> select(_.ENTITY_NAME, _.contains("_FIRST_NAME")) 
        >> pivot_longer(_.contains("_FIRST_NAME"), names_to="position", values_to="first_name")
        >> mutate(position=_.position.str.replace('_FIRST_NAME', '', regex=False))
      )

In [5]:
lns = (scolist 
        >> select(_.ENTITY_NAME, _.contains("_LAST_NAME")) 
        >> pivot_longer(_.contains("_LAST_NAME"), names_to="position", values_to="last_name") 
        >> mutate(position=_.position.str.replace('_LAST_NAME', '', regex=False))
      )

In [6]:
emails = (scolist 
        >> select(_.ENTITY_NAME, _.contains("_EMAIL")) 
        >> pivot_longer(_.contains("_EMAIL"), names_to="position", values_to="email") 
        >> mutate(position=_.position.str.replace('_EMAIL', '', regex=False))
      )

In [7]:
contact_names = fns.merge(lns).reset_index(drop=True)

In [8]:
contacts = contact_names.merge(emails).reset_index(drop=True)

In [9]:
contacts.to_excel("gs://calitp-analytics-data/data-analyses/ntd/sco_rtpa_contacts_long.xlsx", index=False)